In [1]:
import pandas as pd
import pencilbox as pb
import numpy as np
from datetime import datetime


Engine(trino://pulkit.agrawal%40grofers.com@adhoc-trino.analytics.blinkit.in:443/blinkit/default)

In [2]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


# Harmonizing Clinical Concepts

## Repository Overview
This repository contains all the necessary files and instructions required for this problem statement on harmonizing clinical concepts.

## Contents of the Zip Folder
- **`Test.xlsx`**  
  Excel file containing input data. Your predictions will be appended to this file.

- **`snomed_all_data.parquet`**  
  Parquet file containing all target codes and descriptions for the SNOMED CT coding system.

- **`rxnorm_all_data.parquet`**  
  Parquet file containing all target codes and descriptions for the RxNorm coding system.

- **`Column Reference Guide.md`**  
  Markdown file explaining the columns present in the SNOMED CT and RxNorm parquet files.

## Submission Guidelines
To complete your submission:

1. **Modify the `Test.xlsx` file** by appending the following three columns:
   - `Output Coding System`: Either `SNOMEDCT_US` or `RXNORM`
   - `Output Target Code`: Predicted code in **string** format
   - `Output Target Description`: Human-readable description of the predicted code

2. **Your solution in a public code repository.**

3. **Ensure your repository includes:**
   - A clear, concise `README.md` explaining:
     - Your technical solution
     - A short walkthrough of the repository
   - Code that can be easily tested with additional input files


# Columns Reference Guide

---

## 1. `CUI` — Concept Unique Identifier
- **Definition**: A unique identifier assigned to a concept.  
- **Purpose**: Groups together all terms from different vocabularies (SNOMED CT, RxNorm, ICD, etc.) that mean the same thing (are synonymous).  
- **Format**: Always starts with a `"C"` followed by 7 digits (e.g., `C0011849`).  
- **Example**:
  - `C0011849` → *Diabetes Mellitus*  
    - In RxNorm: `Diabetes Mellitus`  
    - In SNOMED CT: `44054006 | Diabetes mellitus (disorder) |`

---

## 2. `System` — Source Vocabulary (SAB)
- **Definition**: Abbreviation for the source vocabulary or terminology that contributed the concept.  
- **Purpose**: Identifies where the term came from (RxNorm, SNOMED CT, MeSH, ICD-10, etc.).  
- **Example**:
  - `RXNORM` → concepts from the RxNorm vocabulary.  
  - `SNOMEDCT_US` → concepts from the U.S. edition of SNOMED CT.  

---

## 3. `TTY` — Term Type
- **Definition**: Describes the role of the term within the source vocabulary.  
- **Purpose**: Differentiates between preferred names, synonyms, codes, ingredients, brand names, etc.  
- **Common Values**:
  - `PT` (SNOMED CT) → Preferred Term.  
  - `SY` (RxNorm, SNOMED CT) → Synonym.  
  - `SCD` (RxNorm) → Semantic Clinical Drug (normalized clinical drug concept).  
  - `BN` (RxNorm) → Brand Name.  
- **Example**:
  - RxNorm:  
    - `SCD` → *Metformin 500 MG Oral Tablet*  
    - `BN` → *Glucophage*  
  - SNOMED CT:  
    - `PT` → *Diabetes mellitus (disorder)*  
    - `SY` → *Sugar diabetes*  

---

## 4. `CODE` — Source Code
- **Definition**: The identifier used **within the source vocabulary** for the concept.  
- **Purpose**: Provides the original vocabulary-specific code.  
- **Format**: Varies by vocabulary.  
- **Example**:
  - RxNorm: `860975` → *Metformin 500 MG Oral Tablet*  
  - SNOMED CT: `44054006` → *Diabetes mellitus (disorder)*  

---

## 5. `STR` — String (Term Name)
- **Definition**: The actual human-readable name of the concept as it appears in the source vocabulary.  
- **Purpose**: Stores the textual representation of the concept.  
- **Example**:
  - RxNorm: `Metformin 500 MG Oral Tablet`  
  - SNOMED CT: `Diabetes mellitus (disorder)`  

---

## 6. `STY` — Semantic Type
- **Definition**: The broad category or semantic group.  
- **Purpose**: Provides higher-level grouping of concepts beyond source vocabularies.  
- **Examples**:
  - `T047` → *Disease or Syndrome*  
  - `T121` → *Pharmacologic Substance*  
- **Example**:
  - `C0011849` (*Diabetes Mellitus*) → `Disease or Syndrome`  
  - `C0025598` (*Metformin*) → `Pharmacologic Substance`  

---

# Summary Table

| Column  | Meaning                                | Example (RxNorm)                   | Example (SNOMED CT)                    |
|---------|----------------------------------------|------------------------------------|----------------------------------------|
| `CUI`   | Concept Unique Identifier              | `C0025598` (*Metformin*)           | `C0011849` (*Diabetes Mellitus*)       |
| `System`| Source vocabulary abbreviation (SAB)   | `RXNORM`                           | `SNOMEDCT_US`                          |
| `TTY`   | Term type in source vocabulary         | `SCD` (Semantic Clinical Drug)     | `PT` (Preferred Term)                  |
| `CODE`  | Code in source vocabulary              | `860975`                           | `44054006`                             |
| `STR`   | Human-readable string/term             | *Metformin 500 MG Oral Tablet*     | *Diabetes mellitus (disorder)*         |
| `STY`   | Semantic type                          | *Pharmacologic Substance*          | *Disease or Syndrome*                  |

---


In [9]:
df = pd.read_parquet('rxnorm_all_data.parquet')

In [10]:
df1 = pd.read_parquet("snomed_all_data.parquet")

In [11]:
df2 = pd.read_excel("Test.xlsx")

In [7]:
pip install thefuzz

  Using cached thefuzz-0.22.1-py3-none-any.whl (8.2 kB)
  Using cached rapidfuzz-3.13.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from thefuzz import fuzz
import re
import time

# --- 1. Load Knowledge Base from External Files ---

def load_correction_map(filepath="correction_map.txt"):
    """Loads the correction map from a text file."""
    correction_map = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                if ':' in line:
                    key, value = line.strip().split(':', 1)
                    correction_map[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Warning: '{filepath}' not found. The cleaner will run without a correction map.")
    return correction_map

def load_redundant_keywords(filepath="redundant_keywords.txt"):
    """Loads the list of redundant keywords from a text file."""
    try:
        with open(filepath, 'r') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"Warning: '{filepath}' not found. The cleaner will run without removing keywords.")
        return []

# --- 2. Advanced Cleaning Pipeline (Now driven by external files) ---

def create_cleaner(correction_map, redundant_keywords):
    """Creates a cleaning function based on the loaded knowledge base."""
    def clean_and_standardize(text):
        if not isinstance(text, str): return ""
        text = text.lower().strip()
        
        # Remove instructions, dosages, etc.
        text = re.sub(r'sig:.*', '', text)
        text = re.sub(r'take \d+(\s*\(.*\))? tablet\(s\)?', '', text)
        text = re.sub(r'\d+(\.\d+)?\s*(mg|ml|mcg|unit|units|g|gram|grams)\b', '', text)
        text = re.sub(r'\b(by|via|every|with|as needed|at bedtime|oral|route|injection|topically)\b', '', text)
        
        # Apply corrections from the loaded map
        for key, value in sorted(correction_map.items(), key=lambda item: len(item[0]), reverse=True):
            text = re.sub(r'\b' + re.escape(key) + r'\b', value, text)
            
        # Remove redundant keywords from the loaded list
        for word in redundant_keywords:
            text = re.sub(r'\b' + word + r'\b', '', text)
            
        # Final cleanup
        text = re.sub(r'[^a-z0-9\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return clean_and_standardize

# --- 3. Load Data and Initialize Cleaner ---
try:
    test_df = df2
    snomed_df = df1
    rxnorm_df = df
    print("All data files loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure all required CSV files are in the directory.")
    exit()

# Load the knowledge base and create the cleaning function
correction_map = load_correction_map()
redundant_keywords = load_redundant_keywords()
cleaner = create_cleaner(correction_map, redundant_keywords)

print("Applying cleaning pipeline to all text data...")
test_df['Cleaned_Description'] = test_df['Input Entity Description'].apply(cleaner)
snomed_df['Cleaned_STR'] = snomed_df['STR'].apply(cleaner)
rxnorm_df['Cleaned_STR'] = rxnorm_df['STR'].apply(cleaner)

# --- 4. Two-Stage Harmonization (Unchanged) ---
def find_best_match(description, entity_type, snomed_data, rxnorm_data, snomed_tfidf, rxnorm_tfidf, snomed_vectorizer, rxnorm_vectorizer):
    is_snomed = entity_type in ['Procedure', 'Lab', 'Diagnosis']
    data, tfidf_matrix, vectorizer, system = (snomed_data, snomed_tfidf, snomed_vectorizer, 'SNOMEDCT_US') if is_snomed else (rxnorm_data, rxnorm_tfidf, rxnorm_vectorizer, 'RXNORM')
    
    if not description.strip():
        return {'System': system, 'Code': 'NO_INPUT', 'Description': 'Original text was empty after cleaning'}

    desc_tfidf = vectorizer.transform([description])
    similarities = cosine_similarity(desc_tfidf, tfidf_matrix).flatten()
    candidate_indices = similarities.argsort()[-20:][::-1]
    
    if len(candidate_indices) == 0 or similarities[candidate_indices[0]] < 0.1:
        return {'System': system, 'Code': 'NOT_FOUND', 'Description': 'No suitable match found'}

    best_final_score, best_match_index = -1, -1
    input_words = set(description.split())
    
    for idx in candidate_indices:
        candidate_str = data.iloc[idx]['Cleaned_STR']
        wratio_score = fuzz.WRatio(description, candidate_str)
        candidate_words = set(candidate_str.split())
        overlap_score = (len(input_words.intersection(candidate_words)) / len(input_words)) * 100 if input_words else 0
        final_score = (wratio_score * 0.4) + (overlap_score * 0.6)
        
        if final_score > best_final_score:
            best_final_score, best_match_index = final_score, idx

    return {'System': system, 'Code': str(data.iloc[best_match_index]['CODE']), 'Description': data.iloc[best_match_index]['STR']}

# --- 5. Execution ---
print("Preparing TF-IDF matrices...")
snomed_vectorizer = TfidfVectorizer(stop_words='english')
snomed_tfidf = snomed_vectorizer.fit_transform(snomed_df['Cleaned_STR'].fillna(''))
rxnorm_vectorizer = TfidfVectorizer(stop_words='english')
rxnorm_tfidf = rxnorm_vectorizer.fit_transform(rxnorm_df['Cleaned_STR'].fillna(''))

print("Running harmonization process...")
start_time = time.time()
results = []
for index, row in test_df.iterrows():
    match = find_best_match(row['Cleaned_Description'], row['Entity Type'], snomed_df, rxnorm_df, snomed_tfidf, rxnorm_tfidf, snomed_vectorizer, rxnorm_vectorizer)
    results.append({'Input Entity Description': row['Input Entity Description'], 'Entity Type': row['Entity Type'], 'Output Coding System': match['System'], 'Output Target Code': match['Code'], 'Output Target Description': match['Description']})

print(f"Harmonization finished in {time.time() - start_time:.2f} seconds.")

# --- 6. Final Output ---
final_df = pd.DataFrame(results)
output_filename = 'final.csv'
final_df.to_csv(output_filename, index=False)


All data files loaded successfully!
Applying cleaning pipeline to all text data...
